In [14]:
!pip install transformers
!pip install emoji
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 15.7 MB/s 
     |████████████████████████████████| 110 kB 71.5 MB/s 


In [25]:
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
import PIL
import PIL.Image
import pandas as pd
from transformers import RobertaTokenizer, TFRobertaModel, RobertaConfig
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_fscore_support
from tensorflow.keras.initializers import TruncatedNormal
from keras import backend as K
from tensorflow.keras.optimizers import Adam
import collections
import re
import unicodedata
import emoji
import contractions
import tokenizers

np.random.seed(42) # note that you must use the same seed to ensure consistentcy in your training/validation/testing
tf.random.set_seed(42)

In [16]:
ds = tfds.load('goemotions')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/43410 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/goemotions/0.1.0.incomplete9WMWI9/goemotions-train.tfrecord*...:   0%|        …

Generating validation examples...:   0%|          | 0/5426 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/goemotions/0.1.0.incomplete9WMWI9/goemotions-validation.tfrecord*...:   0%|   …

Generating test examples...:   0%|          | 0/5427 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/goemotions/0.1.0.incomplete9WMWI9/goemotions-test.tfrecord*...:   0%|         …

Dataset goemotions downloaded and prepared to ~/tensorflow_datasets/goemotions/0.1.0. Subsequent calls will reuse this data.


In [17]:
def remove_multilabels(ds, split):
    df = tfds.as_dataframe(ds[split])
    col_names = [c for c in df.columns if c!="comment_text" ]
    df['sum'] = df[col_names].sum(axis=1)
    df = df[df['sum']==1]
    #ds = tf.data.Dataset.from_tensor_slices(dict(df))
    #return ds
    return df

In [18]:
#subset data to single label entries only
ds_train = remove_multilabels(ds, 'train')
ds_valid = remove_multilabels(ds, 'validation')
ds_test = remove_multilabels(ds, 'test')

In [19]:
#convert byte to string
ds_train['comment_text'] = ds_train['comment_text'].str.decode("utf-8")
ds_valid['comment_text'] = ds_valid['comment_text'].str.decode("utf-8")
ds_test['comment_text'] = ds_test['comment_text'].str.decode("utf-8")

<ipython-input-19-b2b7d861198a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_train['comment_text'] = ds_train['comment_text'].str.decode("utf-8")
<ipython-input-19-b2b7d861198a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_valid['comment_text'] = ds_valid['comment_text'].str.decode("utf-8")
<ipython-input-19-b2b7d861198a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [46]:
GE_taxonomy = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
               'confusion', 'curiosity', 'desire', 'disappointment',
               'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
               'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral',
               'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness',
               'surprise']

In [47]:
nb_labels = len(GE_taxonomy)

In [48]:
full_text = pd.concat([ds_train['comment_text'], ds_test['comment_text'], ds_valid['comment_text']])
MAX_LEN = full_text.apply(lambda x: len(x.split())).max()
MAX_LEN

32

In [49]:
def preprocess_corpus(x):

  # Adding a space between words and punctation
  x = re.sub( r'([a-zA-Z\[\]])([,;.!?])', r'\1 \2', x)
  x = re.sub( r'([,;.!?])([a-zA-Z\[\]])', r'\1 \2', x)
  
  # Demojize
  x = emoji.demojize(x)
  
  # Expand contraction
  x = contractions.fix(x)
  
  # Lower
  x = x.lower()

  # #correct some acronyms/typos/abbreviations  
  # x = re.sub(r"lmao", "laughing my ass off", x)  
  # x = re.sub(r"amirite", "am i right", x)
  # x = re.sub(r"\b(tho)\b", "though", x)
  # x = re.sub(r"\b(ikr)\b", "i know right", x)
  # x = re.sub(r"\b(ya|u)\b", "you", x)
  # x = re.sub(r"\b(eu)\b", "europe", x)
  # x = re.sub(r"\b(da)\b", "the", x)
  # x = re.sub(r"\b(dat)\b", "that", x)
  # x = re.sub(r"\b(dats)\b", "that is", x)
  # x = re.sub(r"\b(cuz)\b", "because", x)
  # x = re.sub(r"\b(fkn)\b", "fucking", x)
  # x = re.sub(r"\b(tbh)\b", "to be honest", x)
  # x = re.sub(r"\b(tbf)\b", "to be fair", x)
  # x = re.sub(r"faux pas", "mistake", x)
  # x = re.sub(r"\b(btw)\b", "by the way", x)
  # x = re.sub(r"\b(bs)\b", "bullshit", x)
  # x = re.sub(r"\b(kinda)\b", "kind of", x)
  # x = re.sub(r"\b(bruh)\b", "bro", x)
  # x = re.sub(r"\b(w/e)\b", "whatever", x)
  # x = re.sub(r"\b(w/)\b", "with", x)
  # x = re.sub(r"\b(w/o)\b", "without", x)
  # x = re.sub(r"\b(doj)\b", "department of justice", x)
  
  # #replace some words with multiple occurences of a letter, example "coooool" turns into --> cool
  # x = re.sub(r"\b(j+e{2,}z+e*)\b", "jeez", x)
  # x = re.sub(r"\b(co+l+)\b", "cool", x)
  # x = re.sub(r"\b(g+o+a+l+)\b", "goal", x)
  # x = re.sub(r"\b(s+h+i+t+)\b", "shit", x)
  # x = re.sub(r"\b(o+m+g+)\b", "omg", x)
  # x = re.sub(r"\b(w+t+f+)\b", "wtf", x)
  # x = re.sub(r"\b(w+h+a+t+)\b", "what", x)
  # x = re.sub(r"\b(y+e+y+|y+a+y+|y+e+a+h+)\b", "yeah", x)
  # x = re.sub(r"\b(w+o+w+)\b", "wow", x)
  # x = re.sub(r"\b(w+h+y+)\b", "why", x)
  # x = re.sub(r"\b(s+o+)\b", "so", x)
  # x = re.sub(r"\b(f)\b", "fuck", x)
  # x = re.sub(r"\b(w+h+o+p+s+)\b", "whoops", x)
  # x = re.sub(r"\b(ofc)\b", "of course", x)
  # x = re.sub(r"\b(the us)\b", "usa", x)
  # x = re.sub(r"\b(gf)\b", "girlfriend", x)
  # x = re.sub(r"\b(hr)\b", "human ressources", x)
  # x = re.sub(r"\b(mh)\b", "mental health", x)
  # x = re.sub(r"\b(idk)\b", "i do not know", x)
  # x = re.sub(r"\b(gotcha)\b", "i got you", x)
  # x = re.sub(r"\b(y+e+p+)\b", "yes", x)
  # x = re.sub(r"\b(a*ha+h[ha]*|a*ha +h[ha]*)\b", "haha", x)
  # x = re.sub(r"\b(o?l+o+l+[ol]*)\b", "lol", x)
  # x = re.sub(r"\b(o*ho+h[ho]*|o*ho +h[ho]*)\b", "ohoh", x)
  # x = re.sub(r"\b(o+h+)\b", "oh", x)
  # x = re.sub(r"\b(a+h+)\b", "ah", x)
  # x = re.sub(r"\b(u+h+)\b", "uh", x)

  # # Handling emojis
  x = re.sub(r"<3", " love_heart ", x)
  x = re.sub(r"xd", " smiling_face_with_open_mouth_and_tightly_closed_eyes ", x)
  x = re.sub(r":\)", " smiling_face ", x)
  x = re.sub(r"^_^", " smiling_face ", x)
  x = re.sub(r"\*_\*", " star_struck ", x)
  x = re.sub(r":\(", " frowning_face ", x)
  x = re.sub(r":\^\(", " frowning_face ", x)
  x = re.sub(r";\(", " frowning_face ", x)
  x = re.sub(r":\/",  " confused_face", x)
  x = re.sub(r";\)",  " wink", x)
  x = re.sub(r">__<",  " unamused ", x)
  x = re.sub(r"\b([xo]+x*)\b", " xoxo ", x)
  x = re.sub(r"\b(n+a+h+)\b", "nah", x)

  # # Handling special cases of text
  # x = re.sub(r"h a m b e r d e r s", "hamberders", x)
  # x = re.sub(r"b e n", "ben", x)
  # x = re.sub(r"s a t i r e", "satire", x)
  # x = re.sub(r"y i k e s", "yikes", x)
  # x = re.sub(r"s p o i l e r", "spoiler", x)
  # x = re.sub(r"thankyou", "thank you", x)
  # x = re.sub(r"a^r^o^o^o^o^o^o^o^n^d", "around", x)

  # Remove special characters and numbers replace by space + remove double space
  x = re.sub(r"\b([.]{3,})"," dots ", x)
  x = re.sub(r"[^A-Za-z!?_]+"," ", x)
  x = re.sub(r"\b([s])\b *","", x)
  x = re.sub(r" +"," ", x)
  x = x.strip()

  return x

In [50]:
ds_train["clean_text"] = ds_train["comment_text"].apply(preprocess_corpus)
ds_valid["clean_text"] = ds_valid["comment_text"].apply(preprocess_corpus)
ds_test["clean_text"] = ds_test["comment_text"].apply(preprocess_corpus)

# Preview of data
display(ds_train[['comment_text', 'clean_text']].sample(5))

,comment_text,clean_text
2104,"Wow your sister is in abusive relationship, co...",wow your sister is in abusive relationship cool
30853,If you're blaming her for the design of the mo...,if you are blaming her for the design of the m...
10854,Being compared to [NAME] is a compliment becau...,being compared to name is a compliment because...
8734,They just dont want to go to school. Source:am...,they just do not want to go to school source a...
5755,I have trouble even remembering what true happ...,i have trouble even remembering what true happ...


In [51]:
# Creating train, validation and test variables
X_train = ds_train['clean_text']
y_train = ds_train.loc[:, GE_taxonomy].values.astype(float)

X_valid = ds_valid['clean_text']
y_valid = ds_valid.loc[:, GE_taxonomy].values.astype(float)

X_test = ds_test['clean_text']
y_test = ds_test.loc[:, GE_taxonomy].values.astype(float)

In [99]:
X_train.shape

(36308,)

In [52]:
y_train.shape

(36308, 28)

In [64]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

config = RobertaConfig.from_pretrained('roberta-base', output_hidden_states=False)

transformer_model = TFRobertaModel.from_pretrained("roberta-base")

# inputs = tokenizer(train_tensor, return_tensors="tf")

# outputs = model(inputs)

# last_hidden_states = outputs.last_hidden_state
# Tokenizing train data
train_token = tokenizer(
    text = X_train.to_list(),
    add_special_tokens = True,
    max_length = MAX_LEN,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing valisation data
valid_token = tokenizer(
    text = X_valid.to_list(),
    add_special_tokens = True,
    max_length = MAX_LEN,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing test data
test_token = tokenizer(
    text = X_test.to_list(),
    add_special_tokens = True,
    max_length = MAX_LEN,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [54]:
train = {'input_ids': train_token['input_ids'], 'attention_mask': train_token['attention_mask'],'token_type_ids': train_token['token_type_ids']}
val = {'input_ids': valid_token['input_ids'], 'attention_mask': valid_token['attention_mask'],'token_type_ids': valid_token['token_type_ids']}
test = {'input_ids': test_token['input_ids'], 'attention_mask': test_token['attention_mask'],'token_type_ids': test_token['token_type_ids']}

In [100]:
# Creating TF tensors
train_tensor = tf.data.Dataset.from_tensor_slices((train, y_train)).shuffle(len(train)).batch(batchsize)
val_tensor = tf.data.Dataset.from_tensor_slices((val, y_valid)).shuffle(len(val)).batch(batchsize)
test_tensor = tf.data.Dataset.from_tensor_slices((test, y_test)).shuffle(len(test)).batch(batchsize)

In [68]:
def create_model(nb_labels):
  #nb_labels is the number of labels in our data

  # Load the MainLayer
  roberta = transformer_model.layers[0]

  # Build the model inputs
  input_ids = layers.Input(shape=(MAX_LEN,), name='input_ids', dtype='int32')
  attention_mask = layers.Input(shape=(MAX_LEN,), name='attention_mask', dtype='int32')
  token_type_ids = layers.Input(shape=(MAX_LEN,), name='token_type_ids', dtype='int32') #original code called this variable token_ids
  inputs = {'input_ids': input_ids, 'attention_mask': attention_mask, 'token_type_ids': token_type_ids}

  # Load the Transformers BERT model as a layer in a Keras model
  roberta_model = roberta(inputs)[1]
  dropout = layers.Dropout(config.hidden_dropout_prob, name='pooled_output')
  pooled_output = dropout(roberta_model, training=False)

  # Then build the model output
  emotion = layers.Dense(units=nb_labels, activation="sigmoid", kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='emotion')(pooled_output) #possibly change activation to softmax
  #emotion = layers.Dense(units=nb_labels, activation="softmax", kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='emotion')(pooled_output) #possibly change activation to softmax
  outputs = emotion

  # And combine it all in a model object
  model = tf.keras.models.Model(inputs=inputs, outputs=outputs, name='BERT_Label')

  return model

In [96]:
train = {'input_ids': train_token['input_ids'], 'attention_mask': train_token['attention_mask'],'token_type_ids': train_token['token_type_ids']}
val = {'input_ids': valid_token['input_ids'], 'attention_mask': valid_token['attention_mask'],'token_type_ids': valid_token['token_type_ids']}
test = {'input_ids': test_token['input_ids'], 'attention_mask': test_token['attention_mask'],'token_type_ids': test_token['token_type_ids']}

In [101]:
train

{'input_ids': <tf.Tensor: shape=(36308, 32), dtype=int32, numpy=
 array([[    0,   405,    16, ...,     1,     1,     1],
        [    0,  9226,    16, ...,     1,     1,     1],
        [    0,   118,  2307, ...,     1,     1,     1],
        ...,
        [    0, 12967,  4420, ...,     1,     1,     1],
        [    0,   118,  1034, ...,     1,     1,     1],
        [    0,   700,    16, ...,     1,     1,     1]], dtype=int32)>,
 'attention_mask': <tf.Tensor: shape=(36308, 32), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(36308, 32), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, .

In [83]:
#hyperparameters
batchsize = 128 #64 #32 #16
learning_rate = 5.e-05
epochs = 12

In [97]:
train_tensor = tf.data.Dataset.from_tensor_slices((train, y_train)).shuffle(len(train)).batch(batchsize)
val_tensor = tf.data.Dataset.from_tensor_slices((val, y_valid)).shuffle(len(val)).batch(batchsize)
test_tensor = tf.data.Dataset.from_tensor_slices((test, y_test)).shuffle(len(test)).batch(batchsize)

In [86]:
# Function for calculating multilabel class weights
def calculating_class_weights(y_true):
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        #compute_class_weight(class_weight, *, classes, y)
        weights[i] = compute_class_weight('balanced', classes = [0.,1.], y = y_true[:, i])
    return weights

class_weights = calculating_class_weights(y_train)

def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        #paper used sigmoid cross entropy loss function to support multi-label classification
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
        #return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*tf.keras.metrics.sparse_categorical_crossentropy(y_true, y_pred), axis=-1)
    
    return weighted_loss


In [102]:
my_model = create_model(nb_labels)

In [103]:
my_model.summary()

Model: "BERT_Label"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 32)]         0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124645632   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]',     

In [107]:
optimizer = Adam(
    learning_rate = learning_rate,
    )

# Set loss
# loss = tf.keras.metrics.CategoricalCrossentropy(
#     name='categorical_crossentropy',
#     dtype=None,
#     from_logits=True,
#     label_smoothing=0
# )
loss = get_weighted_loss(class_weights)

# Compile the model
my_model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = ["accuracy","AUC"] 
    )

# train the model
history = my_model.fit(train_tensor, 
                    epochs = epochs, 
                    #callbacks = [my_callback],
                    validation_data = val_tensor
                    )

Epoch 1/12
1135/1135 [==============================] - 296s 246ms/step - loss: 0.4687 - accuracy: 0.3360 - auc: 0.8533 - categorical_crossentropy: 0.0000e+00 - val_loss: 0.3656 - val_accuracy: 0.4771 - val_auc: 0.9183 - val_categorical_crossentropy: 0.0000e+00
Epoch 2/12
1135/1135 [==============================] - 274s 241ms/step - loss: 0.3291 - accuracy: 0.4614 - auc: 0.9215 - categorical_crossentropy: 0.0000e+00 - val_loss: 0.3317 - val_accuracy: 0.4710 - val_auc: 0.9253 - val_categorical_crossentropy: 0.0000e+00
Epoch 3/12
1135/1135 [==============================] - 273s 240ms/step - loss: 0.2800 - accuracy: 0.4807 - auc: 0.9373 - categorical_crossentropy: 0.0000e+00 - val_loss: 0.3166 - val_accuracy: 0.4809 - val_auc: 0.9361 - val_categorical_crossentropy: 0.0000e+00
Epoch 4/12
1135/1135 [==============================] - 278s 245ms/step - loss: 0.2379 - accuracy: 0.5105 - auc: 0.9501 - categorical_crossentropy: 0.0000e+00 - val_loss: 0.3064 - val_accuracy: 0.4914 - val_auc: 0.

In [108]:
# Making probability predictions on test data
y_pred_proba = my_model.predict(test_tensor)

# from probabilities to labels using a given threshold
def proba_to_labels(y_pred_proba, threshold=0.8):
    
    y_pred_labels = np.zeros_like(y_pred_proba)
    
    for i in range(y_pred_proba.shape[0]):
        for j in range(y_pred_proba.shape[1]):
            if y_pred_proba[i][j] > threshold:
                y_pred_labels[i][j] = 1
            else:
                y_pred_labels[i][j] = 0
                
    return y_pred_labels

# Generate labels
y_pred_labels = proba_to_labels(y_pred_proba)

# Model evaluation function 
def model_eval(y_true, y_pred_labels, emotions):
    
    # Defining variables
    precision = []
    recall = []
    f1 = []
    
    # Per emotion evaluation      
    idx2emotion = {i: e for i, e in enumerate(emotions)}
    
    for i in range(len(emotions)):
   
        # Computing precision, recall and f1-score
        p, r, f1_score, _ = precision_recall_fscore_support(y_true[:, i], y_pred_labels[:, i], average="binary")
        
        # Append results in lists
        precision.append(round(p, 2))
        recall.append(round(r, 2))
        f1.append(round(f1_score, 2))
    
    # Macro evaluation
    macro_p, macro_r, macro_f1_score, _ = precision_recall_fscore_support(y_true, y_pred_labels, average="macro")
    
    # Append results in lists
    precision.append(round(macro_p, 2))
    recall.append(round(macro_r, 2))
    f1.append(round(macro_f1_score, 2))
    
    # Converting results to a dataframe
    df_results = pd.DataFrame({"Precision":precision, "Recall":recall, 'F1':f1})
    df_results.index = emotions+['MACRO-AVERAGE']
    
    return df_results

144/144 [==============================] - 14s 81ms/step


In [109]:
# Model evaluation
model_eval(y_test, y_pred_labels, GE_taxonomy) #for the record, paper had F1 = 0.46

,Precision,Recall,F1
admiration,0.15,0.20,0.17
amusement,0.09,0.13,0.11
anger,0.07,0.14,0.09
annoyance,0.08,0.25,0.12
approval,0.07,0.07,0.07
caring,0.04,0.08,0.05
confusion,0.05,0.20,0.08
curiosity,0.08,0.20,0.12
desire,0.06,0.09,0.07
disappointment,0.02,0.07,0.03
